In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DB_CONN")

In [2]:
pip install "psycopg[binary]" 


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install psycopg

In [ ]:
import psycopg

# establish a connection to the database
conn = psycopg.connect(dbconn)

# or use the connection string directly
# create a cursor object
cur = conn.cursor()

In [ ]:
# Create a table for the stock market data
def create_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS api_data(
                date TIMESTAMP PRIMARY KEY,
                open FLOAT,
                high FLOAT,
                low FLOAT,
                close FLOAT,
                volume FLOAT
                            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_table()


In [ ]:
# Create a table for the article data
def create_article_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS article_data(
                date TIMESTAMP PRIMARY KEY,
                title TEXT,
                author TEXT,
                link TEXT
            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_article_table()

In [ ]:
import pandas as pd

btc_df = pd.read_csv('btc_prices.csv')
btc_df.head()

In [ ]:
# Insert stock price data to the supabase table

from datetime import datetime
import psycopg
import pandas as pd

def insert_stock_data_from_df(df):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    for _, row in df.iterrows():
        # Convert date to datetime if needed
        if isinstance(row['date'], str):
            date_obj = datetime.strptime(row['date'], "%Y-%m-%d")
        else:
            date_obj = pd.to_datetime(row['date'])

        cur.execute(
            '''
            INSERT INTO api_data (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (date) DO NOTHING;
            ''',
            (date_obj, row['open'], row['high'], row['low'], row['close'], row['volume'])
        )

    conn.commit()
    cur.close()
    conn.close()

# Then call it like this:
insert_stock_data_from_df(btc_df)


In [4]:
import pandas as pd
article_df = pd.read_csv('u_today_articles.csv')
article_df.head()

,date,title,author,link
0,"Jun 3, 2025 - 8:26","Tether Moves $1,107,371,328 Bitcoin, CEO Break...",Gamza Khanzadaev,https://u.today/tether-moves-1107371328-bitcoi...
1,"Jun 3, 2025 - 7:29",Gold Copying Bitcoin’s Breakout Trajectory: Le...,Yuri Molchan,https://u.today/gold-copying-bitcoins-breakout...
2,"Jun 3, 2025 - 6:04",Bitcoin ETFs: SEC Commissioner Says Agency to ...,Alex Dovbnya,https://u.today/bitcoin-etfs-sec-commissioner-...
3,"Jun 3, 2025 - 1:13",$5.86 Billion In Bitcoin Flows Into ETFs In May,Caroline Amosun,https://u.today/586-billion-in-bitcoin-flows-i...
4,"Jun 2, 2025 - 23:43",Strategy to Boost Bitcoin (BTC) Acquisitions w...,Paul Adedoyin,https://u.today/strategy-to-boost-bitcoin-btc-...


In [5]:
import psycopg
from datetime import datetime

def insert_article_data_from_df(df):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    for _, row in df.iterrows():
        # Convert date to datetime if needed
        if isinstance(row['date'], str):
            try:
                date_obj = datetime.strptime(row['date'], "%b %d, %Y - %H:%M")
            except ValueError:
                # Try alternative format if needed
                date_obj = pd.to_datetime(row['date'])
        else:
            date_obj = pd.to_datetime(row['date'])

        cur.execute(
            '''
            INSERT INTO article_data (date, title, author, link)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (date) DO NOTHING;
            ''',
            (date_obj, row['title'], row['author'], row['link'])
        )

    conn.commit()
    cur.close()
    conn.close()

insert_article_data_from_df(article_df)